In [2]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 25.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
import selenium
import time

In [89]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

In [14]:
PATH = "/Users/dhairya/Downloads/Application Source Files/chromedriver_mac_arm64/chromedriver.exe"

In [109]:
# You can also optimize the web scraping process by selecting headless option
driver = webdriver.Chrome()
driver.get("https://www.metmuseum.org/art/collection/search?material=Ivory")

In [110]:
titles = driver.find_elements(By.XPATH, "//div[@class='collection-object_title__1MnJJ']")

# Extract all the titles from the museum website
for title in titles:
    print(title.text)

Relief of a head of woman
Figure
Teapot
Sarah Miriam Peale
Horse bit cheekpiece
The Children of Homer Ramsdell, Esq.
Furniture element carved in the round with the head of a roaring lion
Parasol
Clapper
George Washington
Maria Margaretta Oswald
Button or Bead
Spindle Whorl
Netsuke of Two Male Figures
Horse frontlet carved in relief with a female figure flanked by lions
Daniel Strobel, Jr.
Case (Inrō) with Design of Chinese Lady Holding Fan beside a Table (obverse); Low Table with Ceramic Bowl, Stand and Coral Arrangement (reverse)
Snow Goggles
In the Garden
Sword with Scabbard
Fan
Furniture plaque carved in relief with a striding, falcon-headed winged sphinx supported by two kneeling figures
Cricket cage
Mrs. Michael Taney (Monica Brooke)
Tobacco Pouch and Pipe Case
Chess Piece, Probably a Pawn
Souvenir
Bracelet
Souvenir
Tureen and ladle
Button or Bead
Chessman
Furniture element
Button or Bead
Primer
Dr. Henry Nicolls Kitchin
Netsuke
Alexander Petrunkevitch
Portrait of a Woman
Anne Gol

In [111]:
def scrape_item_details(link):
    driver.get(link)
    
    # Wait for the overview section to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "overview")))
    
    # Scrape the product details dynamically
    detail_elements = driver.find_elements(By.XPATH, "//section[@id='overview']//p[@class='artwork-tombstone--item']")
    item_details = {}
    for element in detail_elements:
        label = element.find_element(By.XPATH, ".//span[@class='artwork-tombstone--label']").text.strip(':')
        value = element.find_element(By.XPATH, ".//span[@class='artwork-tombstone--value']").text
        if label in item_details:
            # Handle multiple designers or other repeated attributes
            item_details[label] = f"{item_details[label]}, {value}"
        else:
            item_details[label] = value
    
    return item_details


def scrape_current_page():
    try:
        # Locate the elements using relative XPath for titles
        titles = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//figcaption[@class='collection-object_caption__Wbhum']/div[@class='collection-object_title__1MnJJ']/a[@class='collection-object_link__qM3YR']"))
        )

        for i in range(len(titles)):
            try:
                # Re-locate the title elements before each iteration to avoid stale references
                titles = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.XPATH, "//figcaption[@class='collection-object_caption__Wbhum']/div[@class='collection-object_title__1MnJJ']/a[@class='collection-object_link__qM3YR']"))
                )
                
                title = titles[i]
                item_link = title.get_attribute('href')
                item_details = scrape_item_details(item_link)
                all_items_details.append(item_details)
                print(item_details)
                
                # Navigate back to the main page
                driver.back()
                
                # Add a brief wait to ensure the main page loads properly
                time.sleep(2)
            except StaleElementReferenceException:
                print(f"Encountered a stale element reference exception at index {i}, skipping to next element...")
            except TimeoutException:
                print(f"Encountered a timeout exception at index {i}, skipping to next element...")
            except Exception as e:
                print(f"An unexpected error occurred at index {i}: {e}, skipping to next element...")

    except (StaleElementReferenceException, TimeoutException) as e:
        print(f"Encountered an exception: {e}, attempting to continue...")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Scrape data from all pages
while True:
    scrape_current_page()
    try:
        # Locate and click the "Next Page" button
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Next Page']"))
        )
        next_button.click()
        
        # Wait for the next page to load
        time.sleep(3)
    except TimeoutException:
        # If no next button is found, break the loop
        print("No more pages left to scrape.")
        break
    except Exception as e:
        print(f"An unexpected error occurred while trying to click the next button: {e}")
        break

driver.quit()

{'Title': 'Relief of a head of woman', 'Medium': 'Ivory', 'Dimensions': 'Other: 1 11/16 x 1 1/4 x 1/4 in. (4.3 x 3.2 x 0.7 cm)', 'Classifications': 'Miscellaneous-Bone, Ivory', 'Credit Line': 'Gift of J. Pierpont Morgan, 1917', 'Accession Number': '17.190.72'}
{'Title': 'Figure', 'Date': '2nd–4th century', 'Geography': 'United States, Alaska', 'Culture': 'Punuk', 'Medium': 'Ivory (walrus)', 'Dimensions': 'H.1 11/16 in. (4.3 cm)', 'Classification': 'Bone/Ivory-Sculpture', 'Credit Line': 'Purchase, Bella and Israel Unterberg Foundation Inc. Gift and Funds from various donors, 1981', 'Accession Number': '1981.357'}
{'Title': 'Teapot', 'Maker': 'Tiffany & Co. (1837–present)', 'Date': 'ca. 1887', 'Geography': 'Made in New York, New York, United States', 'Culture': 'American', 'Medium': 'Silver, silver-gilt, enamel and ivory', 'Dimensions': '4 3/4 x 10 7/8 x 5 3/16 in. (12.1 x 27.6 x 13.2 cm); 12 oz. 13 dwt. (393 g)', 'Credit Line': 'Gift of a Friend of the Museum, 1897', 'Accession Number':

From the above output, we see that the data is scraped from the website with all the details. 

Since, this is was a self-learning project, I stopped the process after scraping a couple of pages and hence, you see that "disconnected" error message.

In [113]:
import pandas as pd
df = pd.DataFrame(all_items_details)
df

,Title,Medium,Dimensions,Classifications,Credit Line,Accession Number,Date,Geography,Culture,Classification,Maker,Artist,Period,Dynasty,Rights and Reproduction,Designer,Manufacturer,Retailer,Enameler
0,Relief of a head of woman,Ivory,Other: 1 11/16 x 1 1/4 x 1/4 in. (4.3 x 3.2 x ...,"Miscellaneous-Bone, Ivory","Gift of J. Pierpont Morgan, 1917",17.190.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Figure,Ivory (walrus),H.1 11/16 in. (4.3 cm),NaN,"Purchase, Bella and Israel Unterberg Foundatio...",1981.357,2nd–4th century,"United States, Alaska",Punuk,Bone/Ivory-Sculpture,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Teapot,"Silver, silver-gilt, enamel and ivory",4 3/4 x 10 7/8 x 5 3/16 in. (12.1 x 27.6 x 13....,NaN,"Gift of a Friend of the Museum, 1897",97.1.1,ca. 1887,"Made in New York, New York, United States",American,NaN,Tiffany & Co. (1837–present),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sarah Miriam Peale,watercolor on ivory,3 × 2 1/2 in. (7.6 × 6.4 cm),NaN,"Dale T. Johnson Fund, 2012",2012.558,ca. 1815,NaN,American,NaN,NaN,"James Peale (American, Chestertown, Maryland 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Horse bit cheekpiece,Bone,0.63 x 5.51 in. (1.6 x 14 cm),NaN,"Rogers Fund, 1960",60.20.58,ca. 7th century BCE,"Iran, Hasanlu",Iran,NaN,NaN,NaN,"Iron Age III, Hasanlu Period III",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Souvenir,Gold; ivory,"3 5/16 × 2 in. (8.4 cm);\nminiatures, sight, e...","Metalwork-Gold and Platinum, Miniatures","Gift of J. Pierpont Morgan, 1917",17.190.1261a–c,ca. 1769–70,NaN,"French, Paris",NaN,C. B.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,Netsuke in the form of a Bell with a Boy,Ivory,H. 1 3/8 in. (3.5 cm); W. 1 1/8 in. (2.9 cm); ...,NaN,"Gift of Mrs. Russell Sage, 1910",10.211.1005,18th–19th century,NaN,Japan,Netsuke,NaN,NaN,Edo (1615–1868) or Meiji period (1868–1912),NaN,NaN,NaN,NaN,NaN,NaN
144,Netsuke of Ox with Bokudo,Ivory,H. 1 1/2 in. (3.8 cm); W. 1 3/4 in. (4.4 cm); ...,NaN,"Gift of Mrs. Russell Sage, 1910",10.211.898,late 19th century,NaN,Japan,Netsuke,NaN,NaN,Meiji period (1868–1912),NaN,NaN,NaN,NaN,NaN,NaN
145,Portrait of a Gentleman,Watercolor on ivory,2 11/16 x 2 1/8 in. (6.8 x 5.4 cm),NaN,"Maria DeWitt Jesup Fund, 1992",1992.71,ca. 1825,NaN,American,NaN,NaN,"Nathaniel Rogers (American, Bridgehampton, New...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Save the dataframe as csv
df.to_csv('scraped_data.csv', index=False)